In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import (
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    pipeline, DataCollatorForSeq2Seq, EarlyStoppingCallback
)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# Read the CSV file
data = pd.read_csv("/kaggle/input/english-nagamese-raw-data/raw_data.csv")

In [3]:
# data=data[:100]

In [4]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [5]:
new_lang_token = "<ng>"
tokenizer.add_special_tokens(
    {"additional_special_tokens": [new_lang_token]},
    replace_additional_special_tokens=False
)
model.resize_token_embeddings(len(tokenizer))

M2M100ScaledWordEmbedding(256205, 1024, padding_idx=1)

In [6]:
tokenizer.convert_tokens_to_ids(new_lang_token)

256204

In [7]:
source_texts = list(data["English"])  
target_texts = list(data["Nagamese"]) 



# Split the data into training and validation sets
train_source_texts, val_source_texts, train_target_texts, val_target_texts = train_test_split(
    source_texts, target_texts, test_size=0.20, random_state=40
)


In [8]:
from datasets import Dataset

# Tokenize function
def tokenize_function(examples):
    tokenizer.src_lang = "<en>"
    tokenizer.tgt_lang = "<ng>"
    model_inputs = tokenizer(examples["source_texts"], text_target=examples["target_texts"], return_tensors="pt",max_length=128, truncation=True, padding="max_length")
    return model_inputs

# Convert lists to Hugging Face Dataset
train_dataset = Dataset.from_dict({"source_texts": train_source_texts, "target_texts": train_target_texts})
val_dataset = Dataset.from_dict({"source_texts": val_source_texts, "target_texts": val_target_texts})

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/6360 [00:00<?, ? examples/s]

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

In [9]:
print(train_dataset["input_ids"][0])
print(train_dataset["input_ids"][1])

print(train_dataset["labels"][0])
print(train_dataset["labels"][1])

print(tokenizer.convert_tokens_to_ids("<en>"))
print(tokenizer.convert_tokens_to_ids("<ng>"))

[3, 215246, 39988, 351, 990, 9903, 248079, 1482, 3750, 9713, 21295, 9, 9715, 351, 349, 3422, 248079, 202, 70055, 349, 1537, 156436, 452, 10892, 248075, 57005, 452, 3423, 248079, 117, 259, 1774, 69059, 1738, 248075, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[3, 3291, 248, 324, 1254, 108, 13, 69432, 37, 248280, 796, 248, 172636, 108, 224138, 248075, 3291, 248, 324, 1254, 108, 132362, 14710, 248280, 796, 248, 172636, 108, 30381, 248075, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[256204, 65244, 8118, 100004, 4

In [10]:
!pip install transformers sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.4 MB/s eta 0:00:00


In [11]:
import sacrebleu

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode tokenized outputs into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # SacreBLEU expects reference translations as a **list of lists**
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])

    return {"bleu": bleu.score}

In [12]:
# # Define Data Collator
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps", 
    save_strategy="steps",
    eval_steps=200,
    save_steps=200,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    warmup_steps=200,
    seed = 40,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=25,
    predict_with_generate=True,
    optim="adafactor", 
    report_to=[],
    fp16=True,
    logging_dir="./logs",
    logging_steps=200,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=4)],
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)

# Start Training
trainer.train()


Step,Training Loss,Validation Loss,Bleu
200,7.895200,3.976746,1.964348
400,1.845100,0.666206,14.300833
600,0.593500,0.542657,19.755836
800,0.487700,0.505587,22.369162
1000,0.430600,0.488312,23.522193
1200,0.391300,0.480449,24.089174
1400,0.362200,0.478760,24.306759
1600,0.339700,0.478966,24.458659
1800,0.322600,0.477771,24.754406
2000,0.309700,0.481115,24.374012


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=2475, training_loss=1.1055537414550782, metrics={'train_runtime': 15934.0104, 'train_samples_per_second': 9.979, 'train_steps_per_second': 0.155, 'total_flos': 4.264861856956416e+16, 'train_loss': 1.1055537414550782, 'epoch': 24.753768844221106})

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [14]:
model = model.to(device)

In [15]:
source_texts = val_source_texts
reference_texts = val_target_texts

In [16]:
# Translate source texts using the model
model.eval()
machine_translations = []

for text in source_texts:
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids("<ng>"))
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    machine_translations.append(translated_text)

In [17]:

# Ensure reference_texts is in the required format
wrapped_reference_texts = [reference_texts] 

# Compute BLEU score
bleu = sacrebleu.corpus_bleu(machine_translations, wrapped_reference_texts)
print("BLEU Score:", bleu.score)

BLEU Score: 23.19314279307069
